In [1]:
import pandas as pd

data = pd.read_csv('./5차년도_2차.csv', encoding='CP949')

# 데이터 shape
print(data.shape)
# 전체 감정의 수
emotion_counts = data['상황'].value_counts()
emotion_counts

(19374, 15)
wav_id     0
발화문        0
상황         0
1번 감정      0
1번 감정세기    0
2번 감정      0
2번 감정세기    0
3번 감정      0
3번 감정세기    0
4번 감정      0
4번감정세기     0
5번 감정      0
5번 감정세기    0
나이         0
성별         0
dtype: int64


happiness    4548
angry        3263
neutral      3253
sadness      2848
disgust      2321
surprise     1755
fear         1386
Name: 상황, dtype: int64

##### 아마 5명이 해당 음성에 대해 주관적으로 평가한 것 같음 -> 구체적으로 진행하면 복잡해지기에 우선 상황을 기준을 해당 감정이라고 생각하고 진행 

In [22]:
# 두려움은 약 7%
fear_data = data[data['상황']=='fear']
print(len(fear_data)/len(data))

0.0715391762155466


In [4]:
data['1번 감정'] = data['1번 감정'].apply(str.lower)
data['2번 감정'] = data['2번 감정'].apply(str.lower)
data['3번 감정'] = data['3번 감정'].apply(str.lower)
data['4번 감정'] = data['4번 감정'].apply(str.lower)
data['5번 감정'] = data['5번 감정'].apply(str.lower)

In [9]:
def get_keys(dic): #returns a key for max values in dic
  key_list = list(dic.keys())
  val_list = list(dic.values())
  pos = val_list.index(max(val_list))
  return key_list[pos]
  
final_label = []
for i in range(len(data)):
  sentiments = {'angry':0, 'sadness':0, 'happiness':0, 'fear': 0, 'disgust':0, 'surprise':0, 'neutral':0}
  sentiments[data.iloc[i]['1번 감정']] += data.iloc[i]['1번 감정세기']
  sentiments[data.iloc[i]['2번 감정']] += data.iloc[i]['2번 감정세기']
  sentiments[data.iloc[i]['3번 감정']] += data.iloc[i]['3번 감정세기']
  sentiments[data.iloc[i]['4번 감정']] += data.iloc[i]['4번감정세기']
  sentiments[data.iloc[i]['5번 감정']] += data.iloc[i]['5번 감정세기']

  final_label.append(get_keys(sentiments))
  
final_label_df = pd.DataFrame(final_label, columns=['상황'])
new_data = pd.concat([data[['wav_id', '발화문']], final_label_df], axis = 1)
new_data

,wav_id,발화문,상황
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,happiness
...,...,...,...
19369,5fbe313c44697678c497c05a,나 엘리베이터에 갇혔어.,sadness
19370,5fbe251044697678c497bfb8,하지만 기분이 나쁜 걸 어떡해?,sadness
19371,5fbe31584c55eb78bd7cee7f,자취방 엘리베이턴데 정전인가봐.,sadness
19372,5fbe2f8544697678c497c047,나 드디어 프로젝트 끝났어!,sadness


### 감정 label마다 감정 세기를 더하여 감정 세기가 제일 큰 감정을 해당 발화문의 감정 label로 설정

In [10]:
emotion_counts = new_data['상황'].value_counts()
emotion_counts

sadness      6981
happiness    4710
angry        3804
disgust      1384
surprise     1343
fear         1152
Name: 상황, dtype: int64

### 기존 1386에서 1152로 fear 감소한 것을 알 수 있음

In [11]:
# Separate fear and non-fear samples
fear_samples = new_data[new_data['상황'] == 'fear']
non_fear_samples = new_data[new_data['상황'] != 'fear']

# Randomly sample 1152 non-fear instances
non_fear_sampled = non_fear_samples.sample(n=1152, random_state=42)

# Combine fear and sampled non-fear instances into a new balanced dataset
balanced_data = pd.concat([fear_samples, non_fear_sampled], ignore_index=True)

# Verify the balanced dataset's distribution and size
balanced_data_info = balanced_data['상황'].value_counts(), balanced_data.shape
balanced_data_info

balanced_data

,wav_id,발화문,상황
0,5f414b109dd513131eacee8d,나 아무래도 혼자 갇힌 것 같애.,fear
1,5f414b269dd513131eacee8e,자고 일어났더니 온 마을이 다 물바다야. 나 혼자 있어. 무서워.,fear
2,5f414b409dd513131eacee8f,"내가 잠이 들면 업어가도 몰라. 그래서 재난 문자랑 피하라는 전화도 못 받고, 그냥...",fear
3,5f414b609dd513131eacee90,우리 집 주위에 다 물로 차 있어. 나갈 수가 없어.,fear
4,5f41d9c39dd513131eacef07,나 아무래도 갇힌 거 같아!,fear
...,...,...,...
2299,5fb896864c55eb78bd7cdd41,아예 그냥 사람들도 막 몰려있으니까 짭새가 무슨 일로 떴지? 궁금했어.,angry
2300,5f71d3f2f8fac448cc0a629b,"할 수 없지, 뭐. 언젠간 깨끗해지겠지.",sadness
2301,5fbc8322cb503578af9ee73a,다음에는 더 기록을 단축할려고.,sadness
2302,5f5f8cda3bd6941613f6d3ae,그런 건 아닌 것 같아.,sadness


In [12]:
from sklearn.model_selection import train_test_split

# 불안과 비불안 데이터를 각각 분리
fear_samples = balanced_data[balanced_data['상황'] == 'fear']
non_fear_samples = balanced_data[balanced_data['상황'] != 'fear']

# 각 데이터를 80:20으로 분할
fear_train, fear_test = train_test_split(fear_samples, test_size=0.2, random_state=42)
non_fear_train, non_fear_test = train_test_split(non_fear_samples, test_size=0.2, random_state=42)

# Train과 Test 데이터셋을 결합하여 1:1 비율 유지
train_data = pd.concat([fear_train, non_fear_train], ignore_index=True)
test_data = pd.concat([fear_test, non_fear_test], ignore_index=True)

# 각각의 데이터셋을 CSV 파일로 저장
train_data.to_csv('final_balanced_train_data.csv', index=False, encoding='CP949')
test_data.to_csv('final_balanced_test_data.csv', index=False, encoding='CP949')
# 결과 확인
print("Train 데이터셋 크기:", train_data.shape)
print("Test 데이터셋 크기:", test_data.shape)

Train 데이터셋 크기: (1842, 3)
Test 데이터셋 크기: (462, 3)


In [22]:
import os
import pandas as pd
import shutil

# 경로 설정
test_data_folder = './final_balanced_test_data.csv'
wav_folder = './train_data'
test_folder = './final_test_data'

# train_data 폴더가 없다면 생성
os.makedirs(test_folder, exist_ok=True)

# CSV 파일에서 wav_id 열을 불러오기
data = pd.read_csv(test_data_folder, encoding='CP949')
wav_ids = data['wav_id'].astype(str).tolist()  # 모든 값을 문자열로 변환

# 파일 옮기기
for wav_id in wav_ids:
    wav_file = f"{wav_id}.wav"
    wav_path = os.path.join(wav_folder, wav_file)
    
    # 파일이 존재하면 train_data 폴더로 이동
    if os.path.exists(wav_path):
        shutil.move(wav_path, test_folder)

print("선택된 파일들이 ./test_data 폴더로 이동되었습니다.")


선택된 파일들이 ./test_data 폴더로 이동되었습니다.
